1. Write a Bash script that:
- Creates a directory named "backup" in the user's home directory.
- Copies all .txt files from the current directory into the "backup" directory.
- Appends the current date and time to the filenames of the copied files.

In [ ]:
#!/bin/bash

backup_dir="$HOME/backup"
mkdir -p "$BACKUP_DIR"

# Current timestamp
timestamp=$(date +"%Y-%m-%d_%H-%M-%S")

for file in *.txt; do
  # Handle case where no .txt files exist
  [ -e "$file" ] || continue

  filename="${file%.txt}"
  cp "$file" "$backup_dir/${filename}_$timestamp.txt"
done


2.Create a script that:
- Checks the system’s CPU and memory usage.
- Reports if the CPU usage is above 80% or if the available memory is below 20%.
- Logs the results to a file named system_health.log.

In [ ]:
#!/bin/bash

log_file="system_health.log"
timestamp=$(date +"%Y-%m-%d %H:%M:%S")

# -------- CPU USAGE --------
# Use top in batch mode and extract idle CPU
cpu_idle=$(top -bn1 | grep "Cpu(s)" | awk '{print $8}')
cpu_usage=$(awk "BEGIN {print 100 - $cpu_idle}")

# -------- MEMORY USAGE --------
# Use available memory (not free — free is misleading)
mem_available=$(free | awk '/Mem:/ {print $7}')
total_memory=$(free | awk '/Mem:/ {print $2}')
mem_percent_available=$(awk "BEGIN {print ($mem_available / $total_memory) * 100}")

# -------- STATUS CHECK --------
STATUS="OK"

if (( $(echo "$cpu_usage > 80" | bc -l) )); then
  STATUS="WARNING: High CPU usage"
fi

if (( $(echo "$mem_percent_available < 20" | bc -l) )); then
  STATUS="WARNING: Low available memory"
fi

# -------- LOGGING --------
echo "$timestamp | CPU Usage: ${cpu_usage}% | Available Memory: ${mem_percent_available}% | Status: $STATUS" >> "$log_file"

3. Write a script that:
- Reads a list of usernames from a file (e.g., user_list.txt).
- Creates a new user for each username.
- Generates a random password for each user and saves the username and password to a file named
credentials.txt.

In [ ]:
#!/bin/bash

USER_FILE="user_list.txt"
CRED_FILE="credentials.txt"

# Ensure script is run as root
if [[ $EUID -ne 0 ]]; then
  echo "ERROR: Run this script as root."
  exit 1
fi

# Check if user list exists
if [[ ! -f "$USER_FILE" ]]; then
  echo "ERROR: $USER_FILE not found."
  exit 1
fi

# Clear credentials file
> "$CRED_FILE"
chmod 600 "$CRED_FILE"

# Process each username
while IFS= read -r username; do
  # Skip empty lines or comments
  [[ -z "$username" || "$username" =~ ^# ]] && continue

  # Check if user already exists
  if id "$username" &>/dev/null; then
    echo "User $username already exists. Skipping."
    continue
  fi

  # Generate random password
  password=$(openssl rand -base64 12)

  # Create user
  useradd -m "$username"

  # Set password
  echo "$username:$password" | chpasswd

  # Force password change on first login (BEST PRACTICE)
  chage -d 0 "$username"

  # Save credentials
  echo "$username:$password" >> "$CRED_FILE"

  echo "Created user: $username"
done < "$USER_FILE"

echo "All done. Credentials saved to $CRED_FILE"

4. Create a script that:
- Takes a directory path as input from the user.
- Compresses the directory into a .tar.gz file.
- Saves the compressed file with a name that includes the current date (e.g., backup_2023-08-20.tar.gz).

In [ ]:
#!/bin/bash

# Ask user for directory path
read -p "Enter directory path to back up: " DIR_PATH

# Validate input
if [[ -z "$DIR_PATH" ]]; then
  echo "ERROR: No directory path provided."
  exit 1
fi

if [[ ! -d "$DIR_PATH" ]]; then
  echo "ERROR: '$DIR_PATH' is not a valid directory."
  exit 1
fi

# Extract directory name safely
DIR_NAME=$(basename "$DIR_PATH")

# Current date
DATE=$(date +"%Y-%m-%d")

# Backup file name
ARCHIVE_NAME="backup_${DATE}.tar.gz"

# Create archive
tar -czf "$ARCHIVE_NAME" -C "$(dirname "$DIR_PATH")" "$DIR_NAME"

echo "Backup created successfully: $ARCHIVE_NAME"

5. Create a Bash script that:
- Implements a simple command-line to-do list.
- Allows the user to add tasks, view tasks, and remove tasks.
- Saves the tasks to a file (e.g., todo.txt).

In [ ]:
#!/bin/bash

TODO_FILE="$HOME/todo.txt"

# Ensure todo file exists
touch "$TODO_FILE"

show_menu() {
  echo "----------------------"
  echo " Simple To-Do Manager "
  echo "----------------------"
  echo "1) View tasks"
  echo "2) Add task"
  echo "3) Remove task"
  echo "4) Exit"
}

view_tasks() {
  if [[ ! -s "$TODO_FILE" ]]; then
    echo "No tasks found."
    return
  fi

  echo "Your tasks:"
  nl -w2 -s". " "$TODO_FILE"
}

add_task() {
  read -p "Enter new task: " task
  [[ -z "$task" ]] && echo "Task cannot be empty." && return
  echo "$task" >> "$TODO_FILE"
  echo "Task added."
}

remove_task() {
  view_tasks
  read -p "Enter task number to remove: " task_num

  if ! [[ "$task_num" =~ ^[0-9]+$ ]]; then
    echo "Invalid number."
    return
  fi

  sed -i "${task_num}d" "$TODO_FILE"
  echo "Task removed."
}

while true; do
  show_menu
  read -p "Choose an option: " choice

  case "$choice" in
    1) view_tasks ;;
    2) add_task ;;
    3) remove_task ;;
    4) echo "Goodbye."; exit 0 ;;
    *) echo "Invalid option." ;;
  esac
done

6. Write a script that:
- Reads a list of software package names from a file (e.g., packages.txt).
- Installs each package using the appropriate package manager (apt, yum, etc.).
- Logs the installation status of each package.

In [ ]:
#!/bin/bash

PACKAGE_FILE="packages.txt"
LOG_FILE="install.log"

# Must be run as root
if [[ $EUID -ne 0 ]]; then
  echo "ERROR: Run this script as root."
  exit 1
fi

# Check package list
if [[ ! -f "$PACKAGE_FILE" ]]; then
  echo "ERROR: $PACKAGE_FILE not found."
  exit 1
fi

# Detect package manager
if command -v apt &>/dev/null; then
  PM="apt"
  INSTALL_CMD="apt install -y"
elif command -v yum &>/dev/null; then
  PM="yum"
  INSTALL_CMD="yum install -y"
elif command -v dnf &>/dev/null; then
  PM="dnf"
  INSTALL_CMD="dnf install -y"
else
  echo "ERROR: No supported package manager found."
  exit 1
fi

echo "Using package manager: $PM" | tee -a "$LOG_FILE"
echo "-----------------------------" >> "$LOG_FILE"

# Update package index (important)
if [[ "$PM" == "apt" ]]; then
  apt update >> "$LOG_FILE" 2>&1
fi

# Install packages
while IFS= read -r package; do
  # Skip empty lines and comments
  [[ -z "$package" || "$package" =~ ^# ]] && continue

  echo "Installing $package..."
  if $INSTALL_CMD "$package" >> "$LOG_FILE" 2>&1; then
    echo "$(date '+%Y-%m-%d %H:%M:%S') | $package | SUCCESS" >> "$LOG_FILE"
  else
    echo "$(date '+%Y-%m-%d %H:%M:%S') | $package | FAILED" >> "$LOG_FILE"
  fi
done < "$PACKAGE_FILE"

echo "Installation complete. See $LOG_FILE for details."

7. Create a script that:
- Takes a text file as input.
- Counts and displays the number of lines, words, and characters in the file.
- Finds and displays the longest word in the file.

In [ ]:
#!/bin/bash

# Check input
if [[ $# -ne 1 ]]; then
  echo "Usage: $0 <text-file>"
  exit 1
fi

FILE="$1"

# Validate file
if [[ ! -f "$FILE" ]]; then
  echo "ERROR: File not found."
  exit 1
fi

# Line, word, character counts
LINES=$(wc -l < "$FILE")
WORDS=$(wc -w < "$FILE")
CHARS=$(wc -m < "$FILE")

# Find longest word
LONGEST_WORD=$(tr -c '[:alnum:]' '\n' < "$FILE" | awk '
  length($0) > max {
    max = length($0)
    word = $0
  }
  END { print word }
')

# Output results
echo "File: $FILE"
echo "Lines      : $LINES"
echo "Words      : $WORDS"
echo "Characters : $CHARS"
echo "Longest word: $LONGEST_WORD"